In [1]:
import os
import multiprocessing
import time
from parallel_task_classes import *

if __name__ == '__main__':

    # Define parameters
    # layout_name = "36.png"
    layout_name = "layout"  
    layout_path = os.path.join( os.getcwd(), 'images', layout_name)
    
    # Establish communication queues.
    cell_queue_layout = multiprocessing.Queue()
    partial_cell_queue_layout = multiprocessing.Queue()
    sorted_cell_queue_layout = multiprocessing.Queue()
    

    collector_done_event_layout = multiprocessing.Event()
    row_done_event_layout = multiprocessing.Event()
    
    type_name = '_layout'
    cell_extractor_layout = Cell_extractor(type_name, cell_queue_layout, partial_cell_queue_layout, layout_path, 
                                            row_done_event_layout)
    cell_collector_layout = Cell_collector(type_name, cell_queue_layout, sorted_cell_queue_layout,
                                           row_done_event_layout, collector_done_event_layout)
    cell_merger_layout = Cell_merger(type_name, partial_cell_queue_layout, cell_queue_layout)
    

    SEM_name = "SEM" 
    SEM_path = os.path.join( os.getcwd(), 'images', SEM_name)
    
    # Establish communication queues.
    cell_queue_SEM = multiprocessing.Queue()
    partial_cell_queue_SEM = multiprocessing.Queue()
    sorted_cell_queue_SEM = multiprocessing.Queue()

    collector_done_event_SEM = multiprocessing.Event()
    row_done_event_SEM = multiprocessing.Event()

    type_name = '_SEM'

    cell_extractor_SEM = Cell_extractor(type_name, cell_queue_SEM, partial_cell_queue_SEM, SEM_path, 
                                        row_done_event_SEM)
    cell_collector_SEM = Cell_collector(type_name, cell_queue_SEM, sorted_cell_queue_SEM,
                                           row_done_event_SEM, collector_done_event_SEM)
    cell_merger_SEM = Cell_merger(type_name, partial_cell_queue_SEM, cell_queue_SEM)

    
    cell_validation = Cell_validation(sorted_cell_queue_layout, sorted_cell_queue_SEM, 
                                      collector_done_event_layout, collector_done_event_SEM)

    
    pid_dict ={}

    start = time.time()
    cell_extractor_SEM.start()
    cell_collector_SEM.start()
    cell_merger_SEM.start()
    cell_extractor_layout.start()
    cell_collector_layout.start()
    cell_merger_layout.start()
    cell_validation.start()
    
    # pid_dict['main'] = os.getpid
    pid_dict['cell_extractor_SEM'] = cell_extractor_SEM.pid
    pid_dict['cell_collector_SEM'] = cell_collector_SEM.pid
    pid_dict['cell_merger_SEM'] = cell_merger_SEM.pid
    pid_dict['cell_extractor_layout'] = cell_extractor_layout.pid
    pid_dict['cell_collector_layout'] = cell_collector_layout.pid
    pid_dict['cell_merger_layout'] = cell_merger_layout.pid
    pid_dict['cell_validation'] = cell_validation.pid
    
    program_done_event = multiprocessing.Event()
    resource_logger = multiprocessing.Process(target=log_resource_usage, args=(pid_dict,program_done_event))
    resource_logger.start()

    cell_extractor_SEM.join()
    cell_collector_SEM.join()
    cell_merger_SEM.join()
    cell_extractor_layout.join()
    cell_collector_layout.join()
    cell_merger_SEM.join()
    cell_validation.join()
    end = time.time()

    program_done_event.set()
    resource_logger.join()

    
    print(f"Total execution time = {end-start}")
    

    


KeyboardInterrupt: 


Cell_extractor-1_layout Total cells extracted 153117
Cell_merger-3_layout total cells merged = 0


Cell_extractor-4_SEM Total cells extracted 153117
 Cell_collector-2_layout is now exiting, total cells collected = 153117

Cell_merger-6_SEM total cells merged = 0

 Cell_collector-5_SEM is now exiting, total cells collected = 153117

